# Predicting NBA Games with Machine Learning
## Author: Kishore Annambhotla

### Introduction and Methods

As a math-minded person, I've always been interested in the use of numbers and analytics in sports. Being an avid follower of the NBA, I decided to pursue a project I've thought about for a while: **a machine learning model that could accurately predict the outcomes of NBA regular season games.** This project was my first real foray into data science, machine learning, and sports analytics, and was certainly one of the most instructive things I've ever worked on. The end result of this project is a high-accuracy ML model capable of intelligently predicting NBA games, and this notebook will step through the code and methods I used to create the model.


### Setting Up

In order to predict NBA game outcomes, we will need to analyze many past games and see what statistics help or hurt a team's chances of winning. We will also need a robust predictive model capable of making a decision based on all of this game data. For this project, I used `nba_api` to retrieve up-to-date NBA game data, `pandas` and `numpy` for data manipulation, and `scikit-learn` to build an ML model. All of the project's necessary imports will be included in the code block below.

In [26]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

pd.options.mode.chained_assignment = (
    None  # prevents warnings on reformatted columns later
)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

### Importing Data

We will base our model off of games played by all NBA teams. As shown, this game data will include categorical information about the game, teams, and season, quantitative data regarding one team's offensive and defensive statistics, and of course the winner of the game.

First, we will use the `nba_api` to get games played by each NBA team. Note that we set `season_type_nullable` to only include regular season games, since those are the types of games we aim to predict with this model. This could easily be changed to train a model better for predicting playoff or preseason games.

In [27]:
nba_teams = teams.get_teams()
team_abbr_to_id = {team["abbreviation"]: team["id"] for team in nba_teams}
all_games = pd.DataFrame()

for team in nba_teams:
    team_id = team["id"]
    gamefinder = leaguegamefinder.LeagueGameFinder(
        team_id_nullable=team_id, season_type_nullable="Regular Season"
    )
    games = gamefinder.get_data_frames()[0]
    all_games = pd.concat([all_games, games], ignore_index=True)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [3]:
print(all_games.columns)

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')


In [ ]:
all_games.sample(n=5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
51504,21992,1610612752,NYK,New York Knicks,0029200719,1993-02-28,NYK @ NJN,L,240,76,...,0.800,14.0,22.0,36.0,13,10.0,4,14,28,NaN
17797,21997,1610612742,DAL,Dallas Mavericks,0029700434,1998-01-02,DAL @ PHX,L,239,85,...,0.737,12.0,28.0,40.0,16,14.0,4,20,17,-7.0
27235,22007,1610612745,HOU,Houston Rockets,0020701065,2008-03-26,HOU vs. MIN,W,240,97,...,0.765,20.0,38.0,58.0,22,11.0,2,12,14,11.0
44306,22003,1610612750,MIN,Minnesota Timberwolves,0020300781,2004-02-19,MIN vs. SAC,W,240,92,...,0.792,14.0,46.0,60.0,21,2.0,4,16,22,17.0
15365,21985,1610612741,CHI,Chicago Bulls,0028500114,1985-11-15,CHI @ MIL,L,240,103,...,0.778,18.0,23.0,41.0,26,4.0,3,22,23,NaN


While this data is good, it may be too much for our model. Over the last forty seasons, NBA basketball has greatly changed. Many rules have been added and removed, and aspects of "perfect" offense and defense have completely shifted. As a result, older data is increasingly unreliable for predicting current-day NBA games. For this model, I chose to only use data from the **2019-2020 season onward.** In my testing, this period resulted in the highest accuracy.

In [5]:
# Get games where season is in modern_seasons
games_modern = all_games[
    (all_games.SEASON_ID.str[-4:] == "2019")
    | (all_games.SEASON_ID.str[-4:] == "2020")
    | (all_games.SEASON_ID.str[-4:] == "2021")
    | (all_games.SEASON_ID.str[-4:] == "2022")
    | (all_games.SEASON_ID.str[-4:] == "2023")
    | (all_games.SEASON_ID.str[-4:] == "2024")
]

### Cleaning Data

We have plenty of good data, but it's still unrefined and hard to work with. The next step is to clean up our columns and reformat the data. This will make training, testing, and evaluating our model much easier in the future.

First, we will reformat some of our columns to make working with the data easier.

In [6]:
# Convert GAME_DATE to pandas datetime
# Also order by date earliest to latest to make working with stats easier later
games_modern["GAME_DATE"] = pd.to_datetime(games_modern["GAME_DATE"])
games_modern.sort_values(by="GAME_DATE", inplace=True)


# Add binary "WIN" column, remove categorical WL column
games_modern["WIN"] = games_modern["WL"].apply(lambda x: 1 if x == "W" else 0)


# Convert int stat columns to float type for accurate data analysis
games_modern["MIN"] = games_modern["MIN"].astype(float)  # minutes
games_modern["PTS"] = games_modern["PTS"].astype(float)  # points
games_modern["FGM"] = games_modern["FGM"].astype(float)  # field goals made
games_modern["FGA"] = games_modern["FGA"].astype(float)  # field goals attempted
games_modern["FG3M"] = games_modern["FG3M"].astype(float)  # 3s made
games_modern["FG3A"] = games_modern["FG3A"].astype(float)  # 3s attempted
games_modern["FTM"] = games_modern["FTM"].astype(float)  # free throws made
games_modern["FTA"] = games_modern["FTA"].astype(float)  # free throws attempted
games_modern["REB"] = games_modern["REB"].astype(float)  # rebounds
games_modern["OREB"] = games_modern["OREB"].astype(float)  # offensive rebounds
games_modern["DREB"] = games_modern["DREB"].astype(float)  # defensive rebounds
games_modern["AST"] = games_modern["AST"].astype(float)  # assists
games_modern["BLK"] = games_modern["BLK"].astype(float)  # blocks
games_modern["TOV"] = games_modern["TOV"].astype(float)  # turnovers
games_modern["PF"] = games_modern["PF"].astype(float)  # personal fouls


# Add opponent id as column
def get_opponent_id(matchup, team_abbr_to_id, team_id):
    if "@" in matchup:
        opponent_abbr = matchup.split(" @ ")[-1]
    else:
        opponent_abbr = matchup.split(" vs. ")[-1]
    return team_abbr_to_id.get(opponent_abbr, team_id)


games_modern["OPP_TEAM_ID"] = games_modern.apply(
    lambda row: get_opponent_id(row["MATCHUP"], team_abbr_to_id, row["TEAM_ID"]), axis=1
)

Our data is now workable and could already be used to create a decent model. However, a few new statistics will greatly improve our model's accuracy. First, we can indicate to our model if a team had home game advantage, which can have an immense impact on the away team. Additionally, we can indicate to the model the outcome of the team's last game, as a prior win or loss could affect the team's energy going into a future game. We can then calculate the important **"Four Factors"** in basketball, created by basketball statistician Dean Oliver. These statistics track a team's shooting, turnovers, rebounding, and free throws, and are arguably the best metrics for predicting team success in basketball. Finally, we can calculate a team's true shooting percentage in each game, another intelligent measure of shooting efficiency.

In [20]:
# Define 'HGA' (Home Game Advantage)
games_modern["HGA"] = games_modern["MATCHUP"].apply(lambda x: 0 if "@" in x else 1)

# Define 'LAST_GAME_OUTCOME'
games_modern["LAST_GAME_OUTCOME"] = (
    games_modern.groupby("TEAM_ID")["WIN"].shift(1).fillna(0)
)

# Define 'EFG%' (Effective Field Goal Percentage)
games_modern["EFG%"] = (
    games_modern["FGM"] + (0.5 * games_modern["FG3M"])
) / games_modern["FGA"]

# Define 'TOV%' (Turnover Percentage)
games_modern["TOV%"] = games_modern["TOV"] / (
    games_modern["FGA"] + 0.44 * games_modern["FTA"] + games_modern["TOV"]
)

# Define 'FTR' (Free Throw Attempt Rate)
games_modern["FTR"] = games_modern["FTA"] / games_modern["FGA"]

# Define 'TS%' (True Shooting Percentage)
games_modern["TS%"] = games_modern["PTS"] / (
    2 * (games_modern["FGA"] + (0.44 * games_modern["FTA"]))
)

Finally, we want to use a `LabelEncoder` to transform a few of our categorical data columns into numerical values that can be understood by our model.

In [ ]:
# LabelEncode data
le = LabelEncoder()
games_modern["TEAM_ID"] = le.fit_transform(games_modern["TEAM_ID"])
games_modern["OPP_TEAM_ID"] = le.fit_transform(games_modern["OPP_TEAM_ID"])
games_modern["GAME_ID"] = le.fit_transform(games_modern["GAME_ID"])
games_modern.sample(5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PLUS_MINUS,WIN,OPP_TEAM_ID,HGA,LAST_GAME_OUTCOME,EFG%,TOV%,ORB%,FTR,TS%
29433,22022,9,LAC,LA Clippers,2717,2022-12-12,LAC vs. BOS,W,241.0,113.0,...,20.0,1,1,1,1.0,0.544444,0.083149,0.275000,0.233333,0.569327
72418,22023,22,SAS,San Antonio Spurs,3966,2023-12-28,SAS @ POR,W,241.0,118.0,...,13.0,1,20,0,0.0,0.541176,0.101551,0.194444,0.329412,0.606247
12322,22022,4,CHI,Chicago Bulls,2794,2022-12-23,CHI @ NYK,W,240.0,118.0,...,1.0,1,15,0,1.0,0.559140,0.038124,0.275000,0.193548,0.584621
25940,22023,8,HOU,Houston Rockets,3763,2023-11-20,HOU @ GSW,L,239.0,116.0,...,-5.0,0,7,0,0.0,0.531579,0.071839,0.340909,0.200000,0.561146
36255,22022,11,MIA,Miami Heat,3267,2023-03-04,MIA vs. ATL,W,240.0,117.0,...,8.0,1,0,1,0.0,0.584337,0.169461,0.292683,0.277108,0.628222


We will also now define a dictionary keying each team to their encoded ID. This may not seem useful yet, but will be helpful for data manipulation later.

In [10]:
# Will be helpful to have dictionary keying teams to encoded id values
ENCODED_TEAM_IDS = {
    "Atlanta Hawks": 0,
    "Hawks": 0,
    "Boston Celtics": 1,
    "Celtics": 1,
    "Cleveland Cavaliers": 2,
    "Cavaliers": 2,
    "New Orleans Pelicans": 3,
    "Pelicans": 3,
    "Chicago Bulls": 4,
    "Bulls": 4,
    "Dallas Mavericks": 5,
    "Mavericks": 5,
    "Denver Nuggets": 6,
    "Nuggets": 6,
    "Golden State Warriors": 7,
    "Warriors": 7,
    "Houston Rockets": 8,
    "Rockets": 8,
    "LA Clippers": 9,
    "Clippers": 9,
    "Los Angeles Lakers": 10,
    "Lakers": 10,
    "Miami Heat": 11,
    "Heat": 11,
    "Milwaukee Bucks": 12,
    "Bucks": 12,
    "Minnesota Timberwolves": 13,
    "Timberwolves": 13,
    "Brooklyn Nets": 14,
    "Nets": 14,
    "New York Knicks": 15,
    "Knicks": 15,
    "Orlando Magic": 16,
    "Magic": 16,
    "Indiana Pacers": 17,
    "Pacers": 17,
    "Philadelphia 76ers": 18,
    "76ers": 18,
    "Phoenix Suns": 19,
    "Suns": 19,
    "Portland Trail Blazers": 20,
    "Trail Blazers": 20,
    "Sacramento Kings": 21,
    "Kings": 21,
    "San Antonio Spurs": 22,
    "Spurs": 22,
    "Oklahoma City Thunder": 23,
    "Thunder": 23,
    "Toronto Raptors": 24,
    "Raptors": 24,
    "Utah Jazz": 25,
    "Jazz": 25,
    "Memphis Grizzlies": 26,
    "Grizzlies": 26,
    "Washington Wizards": 27,
    "Wizards": 27,
    "Detroit Pistons": 28,
    "Pistons": 28,
    "Charlotte Hornets": 29,
    "Hornets": 29,
}

### Defining Features

In our model, `X` includes features of the dataset that will be used to predict `y`. Here, `y` is obviously `WIN`, since we are just predicting and classifying each game as a win (1) or loss (0). `X` includes a number of valuable statistics, and all the ones used will be included in the code segment.

We will also use `train_test_split` from `scikit-learn` to train and test our model with an 80-20 split, widely regarded as the recommended training split for most ML models.

In [11]:
features = [
    "TEAM_ID",
    "OPP_TEAM_ID",
    "PTS",
    "OREB",
    "DREB",
    "REB",
    "AST",
    "STL",
    "BLK",
    "TOV",
    "EFG%",
    "TOV%",
    "FTR",
    "TS%",
    "HGA",
    "LAST_GAME_OUTCOME",
]
X = games_modern[features]

y = games_modern[["WIN"]]


# Use sklearn train_test_split with 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Creating Initial Model

`scikit-learn` offers many options for ML classification models. A simple one good for this problem would be a `RandomForestClassifier`, which uses many decision trees to analyze data and settle on a choice. Random forests are known for accuracy, efficiency, and ease of use, making this model a good choice for us.

In [ ]:
rf_model = RandomForestClassifier(n_estimators=175, max_depth=25, random_state=42)
rf_model.fit(X_train, y_train)

c:\Users\14846\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=25, n_estimators=175, random_state=42)

In [28]:
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8202002612102742
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1115
           1       0.84      0.80      0.82      1182

    accuracy                           0.82      2297
   macro avg       0.82      0.82      0.82      2297
weighted avg       0.82      0.82      0.82      2297



### Predicting Games

Our model is accurate, but there's an issue: We can only predict games that have already been played. That's  not very useful and, more importantly, isn't a test of how accurate our model is at predicting games that have **not yet been played.**. However, if we can use our model with **average** stats from teams this season, we can make individual predictions on games to be played in the future. To do this, we will need to know what games are going to be played. Once again, using `nba_api`, we can retrieve games played today with the `scoreboard` object.

In [14]:
# Note: This code comes from the nba_api repository via GitHub. 
# It is used to demonstrate the functionality of the scoreboard.
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}"

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
game_teams = []
for game in games:
    gameTimeLTZ = (
        parser.parse(game["gameTimeUTC"])
        .replace(tzinfo=timezone.utc)
        .astimezone(tz=None)
    )
    print(
        f.format(
            gameId=game["gameId"],
            awayTeam=game["awayTeam"]["teamName"],
            homeTeam=game["homeTeam"]["teamName"],
            gameTimeLTZ=gameTimeLTZ,
        )
    )
    game_teams.append([game["homeTeam"]["teamName"], game["awayTeam"]["teamName"]])

ScoreBoardDate: 2025-01-19
0022400592: Spurs vs. Heat @ 2025-01-19 15:00:00-05:00
0022400593: Nuggets vs. Magic @ 2025-01-19 18:00:00-05:00
0022400594: 76ers vs. Bucks @ 2025-01-19 19:00:00-05:00
0022400595: Nets vs. Thunder @ 2025-01-19 19:00:00-05:00
0022400596: Lakers vs. Clippers @ 2025-01-19 21:00:00-05:00
0022400597: Bulls vs. Trail Blazers @ 2025-01-19 21:00:00-05:00
0022400598: Wizards vs. Kings @ 2025-01-19 21:00:00-05:00


For our data, instead of using games that have already been played, we will get per game and average stats for each team this season, which are most relevant to the games that will be played.

In [15]:
from nba_api.stats.endpoints import leaguedashteamstats

advancedGamefinder = leaguedashteamstats.LeagueDashTeamStats(
    season="2024-25", per_mode_detailed="PerGame"
)

team_stats = advancedGamefinder.get_data_frames()[0]

# Drop all columns related to ranks (not helpful for database)
keyword = "RANK"
columns_to_drop = [col for col in team_stats.columns if keyword in col]
team_stats.drop(columns=columns_to_drop, inplace=True)

Next, we can clean and modify our `team_stats` dataframe. This process is quite similar to the cleaning process for `games_modern`, since we need to match the columns in each dataframe.

In [16]:
# Cleaning data in team_stats and matching to games_modern

# Define three of four factors (EFG%, TOV%, FTR)

# Define 'EFG%' (Effective Field Goal Percentage)
team_stats["EFG%"] = (team_stats["FGM"] + (0.5 * team_stats["FG3M"])) / team_stats[
    "FGA"
]

# Define 'TOV%' (Turnover Percentage)
team_stats["TOV%"] = team_stats["TOV"] / (
    team_stats["FGA"] + 0.44 * team_stats["FTA"] + team_stats["TOV"]
)

# Define 'FTR' (Free Throw Attempt Rate)
team_stats["FTR"] = team_stats["FTA"] / team_stats["FGA"]

# Define 'TS%' (True Shooting Percentage)
team_stats["TS%"] = team_stats["PTS"] / (
    2 * (team_stats["FGA"] + (0.44 * team_stats["FTA"]))
)

# Label encode team ids
team_stats["TEAM_ID"] = le.fit_transform(team_stats["TEAM_ID"])

Finally, with our data cleaned, we can iterate through the games that will be played today and construct a one-row dataframe for each game with the necessary columns and data. Notice that this is also where our dictionary of team IDs comes in handy.

In [17]:
# Fetch data to predict upcoming games
games = board.games.get_dict()
# Empty list to store each prediction dataframe
prediction_dfs = []
for game in games:

    # Get home team (team) and away team (opponent) encoded ids
    team_id = ENCODED_TEAM_IDS.get(game["homeTeam"]["teamName"])
    opp_team_id = ENCODED_TEAM_IDS.get(game["awayTeam"]["teamName"])

    # Get relevant home team statistics for features

    home_condition = team_stats["TEAM_ID"] == team_id

    pts = team_stats.loc[home_condition, "PTS"].values[0]

    oreb = team_stats.loc[home_condition, "OREB"].values[0]

    dreb = team_stats.loc[home_condition, "DREB"].values[0]

    reb = team_stats.loc[home_condition, "REB"].values[0]

    ast = team_stats.loc[home_condition, "AST"].values[0]

    stl = team_stats.loc[home_condition, "STL"].values[0]

    blk = team_stats.loc[home_condition, "BLK"].values[0]

    tov = team_stats.loc[home_condition, "TOV"].values[0]

    efg_pct = team_stats.loc[home_condition, "EFG%"].values[0]

    tov_pct = team_stats.loc[home_condition, "TOV%"].values[0]

    ftr = team_stats.loc[home_condition, "FTR"].values[0]

    ts_pct = team_stats.loc[home_condition, "TS%"].values[0]

    # Get home game advantage (always 1 due to how we format data)
    hga = 1.0

    # Get last game outcome
    filtered_games_modern = games_modern.loc[games_modern["TEAM_ID"] == team_id]
    filtered_games_modern.sort_values(by="GAME_DATE", ascending=False, inplace=True)
    last_game_outcome = filtered_games_modern.iloc[0]["WIN"]

    # With all of the data, construct a dataframe with the game's information.
    prediction_data = {
        "TEAM_ID": [team_id],
        "OPP_TEAM_ID": [opp_team_id],
        "PTS": [pts],
        "OREB": [oreb],
        "DREB": [dreb],
        "REB": [reb],
        "AST": [ast],
        "STL": [stl],
        "BLK": [blk],
        "TOV": [tov],
        "EFG%": [efg_pct],
        "TOV%": [tov_pct],
        "FTR": [ftr],
        "TS%": [ts_pct],
        "HGA": [hga],
        "LAST_GAME_OUTCOME": [last_game_outcome],
    }
    game_prediction_df = pd.DataFrame(prediction_data)
    prediction_dfs.append(game_prediction_df)

# Will all game dataframes, concatenate into a single dataframe for easy simultaneous viewing
all_game_predictions = pd.concat(prediction_dfs)

In [18]:
all_game_predictions

,TEAM_ID,OPP_TEAM_ID,PTS,OREB,DREB,REB,AST,STL,BLK,TOV,EFG%,TOV%,FTR,TS%,HGA,LAST_GAME_OUTCOME
0,11,22,111.2,9.3,34.2,43.5,26.3,8.6,3.8,13.5,0.540276,0.122638,0.253165,0.575689,1.0,1
0,16,6,104.3,11.0,30.4,41.3,23.4,9.2,6.5,15.0,0.508803,0.136022,0.268779,0.547357,1.0,0
0,12,18,113.8,8.5,35.6,44.1,24.8,7.7,5.2,14.2,0.564815,0.128763,0.254630,0.592215,1.0,1
0,23,14,116.2,10.1,33.6,43.7,26.0,11.5,5.8,12.1,0.546790,0.107647,0.207835,0.579239,1.0,0
0,9,10,109.6,10.5,33.9,44.4,24.3,9.8,4.5,16.4,0.540888,0.146964,0.254673,0.575679,1.0,1
0,20,4,107.8,12.5,30.6,43.1,23.3,8.0,5.6,16.5,0.515118,0.143813,0.227324,0.548701,1.0,0
0,21,27,116.5,10.8,33.8,44.7,26.5,8.1,4.7,13.2,0.542857,0.116001,0.239560,0.579072,1.0,1


With each of our games stored in the array `prediction_dfs`, we can finally iterate through and apply our model to each one-row dataframe. In this case, a prediction of `[0]` indicates a loss for the home team and a prediction of `[1]` indicates a victory for the home team.

In [ ]:
for i in range(0, len(prediction_dfs)):
    print("Home Team:", game_teams[i][0])
    print("Away Team:", game_teams[i][1])
    prediction = rf_model.predict(prediction_dfs[i])
    # 0 = loss for home team/win for away team
    # 1 = win for home team/loss for away team
    if prediction == 0:
        print(game_teams[i][1] + " predicted to win!")
    if prediction == 1:
        print(game_teams[i][0] + " predicted to win!")
    print("")

Home Team: Heat
Away Team: Spurs
Heat predicted to win!

Home Team: Magic
Away Team: Nuggets
Nuggets predicted to win!

Home Team: Bucks
Away Team: 76ers
Bucks predicted to win!

Home Team: Thunder
Away Team: Nets
Thunder predicted to win!

Home Team: Clippers
Away Team: Lakers
Clippers predicted to win!

Home Team: Trail Blazers
Away Team: Bulls
Bulls predicted to win!

Home Team: Kings
Away Team: Wizards
Kings predicted to win!



### Conclusions and Future

Overall, I'm proud of how this project turned out. This project really sparked my interest in data science and machine learning, and I would definitely be happy to work in the field of AI/ML in the future. That being said, there are certainly a number of improvements that could be made. One limitation of the data from the `nba_api` was that I could not easily access game stats for the opposing team, potentially limiting the accuracy of my model. This could be fixed in the future by using a web scraper, which would theoretically allow me to retrieve any data I need related to each game played. 

Another limitation of my model is the fact that injuries are not accounted for, which can definitely make some predictions highly inaccurate. This could be fixed by making a model that instead analyzes data related to individual players rather than entire teams. That way, injured players could simply be ignored when comparing teams. In the future, I would like to recreate this project to address these limitations and make an even stronger model.